In [1]:
import numpy as np
from fixedpoint import FixedPoint
import pandas as pd

/tmp/ipykernel_30629/3179091947.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
samples = pd.read_csv('samples.csv')

real_samples = samples[::2].values.reshape(1,int(len(samples)/2))[0]
imag_samples = samples[1::2].values.reshape(1,int(len(samples)/2))[0]

In [3]:
def fixed_point(bits, signed=1, m=17, n=16):
    return float(FixedPoint('0x'+ bits, signed, m, n))

In [4]:
def butterfly(xev_r, xev_i, xod_r, xod_i, N):

    k = np.arange(0, N/2)

    wr = np.cos(-2*np.pi*k/N)
    wi = np.sin(-2*np.pi*k/N)

    g1 = wr*xod_r - wi*xod_i
    g2 = wr*xod_i + wi*xod_r

    y_re = np.concatenate((xev_r + g1, xev_r - g1))
    y_im = np.concatenate((xev_i + g2, xev_i - g2))

    return y_re, y_im

def matrix_computation(x_re, x_im):
    N = len(x_re)

    k = np.arange(0, N)
    
    kj = np.kron(k, k)

    F_re = np.cos(-2*np.pi * kj / N)
    F_im = np.sin(-2*np.pi * kj / N)

    y_re = np.sum(F_re*x_re - F_im*x_im)
    y_im = np.sum(F_re*x_im - F_im*x_re)

    return y_re, y_im

def FFT(x_re, x_im):
    N = len(x_re)
    
    if N % 2 == 0:
        xev_r = x_re[::2]
        xev_i = x_im[::2]

        xod_r = x_re[1::2]
        xod_i = x_im[1::2]

        Xev_r, Xev_i = FFT(xev_r, xev_i)
        Xod_r, Xod_i = FFT(xod_r, xod_i)

        yr, yi = butterfly(Xev_r, Xev_i, Xod_r, Xod_i, N)
    else:
        yr, yi = matrix_computation(x_re, x_im)

    return yr, yi

def iFFT(x_re, x_im):
    N = len(x_re)
    Xre, Xim    = FFT(x_re, x_im)

    return Xre/N, Xim/N



In [5]:
def fixed_point_ftoh(float_nm, signed=1, m=20, n=16):
    return str(FixedPoint(float_nm, signed, m, n, str_base=16))

In [6]:
aux=0
results = list()

while aux < len(real_samples):
    x_real = real_samples[aux:aux+4]
    x_imag = imag_samples[aux:aux+4]

    x_real_decimal = list(map(fixed_point, x_real))
    x_imag_decimal = list(map(fixed_point, x_imag))

    y_r, y_i = FFT(x_real_decimal, x_imag_decimal)
    results.append(tuple(zip(y_r, y_i)))
    aux=aux+4

print(results)

[((-139696.74394226074, -114556.30125427246), (-1989.6705017089807, -13176.426055908203), (-10084.309341430664, -58601.814193725586), (36474.14352416992, 86741.48718261719)), ((-86732.06999206543, -69386.16845703125), (12373.827667236328, 23459.897201538086), (-80453.07273864746, -28791.536224365234), (-9561.383850097656, 21729.04624938965)), ((-184764.98405456543, -34900.01205444336), (69055.30128479004, 10126.872650146484), (47113.182876586914, -8974.54507446289), (45271.05744934082, 11585.484161376953)), ((-155525.58557128906, -91087.75601196289), (24774.489013671875, -49517.60363769531), (-52204.45275878906, -51971.92691040039), (19057.245056152344, -51883.82873535156)), ((-70496.07508850098, -161595.77000427246), (14665.273651123045, 18728.699676513672), (-15178.843551635742, -32915.92930603027), (13226.681610107424, -34976.78649902344)), ((-136854.18716430664, -185828.49702453613), (-28198.815536499023, 15997.850677490234), (21307.604064941406, 29156.908981323242), (-9854.5998992

In [7]:
hex_result = list()
for fft_result in results:
    for c_number in fft_result:
        hex_result.append(fixed_point_ftoh(c_number[0]))
        hex_result.append(fixed_point_ftoh(c_number[1]))

In [8]:
for word in hex_result:
    print(word)

dde4f418d
e4083b2e1
ff83a545a
fcc8792ee
fd89bb0cf
f1b162f91
08e7a24be
152d57cb8
ead33ee15
ef0f5d4e0
03055d3e2
05ba3e5af
ec5baed61
f8f8876ba
fdaa69dbc
054e10bd7
d2e430415
f77abfcea
10dbf4d21
0278edf66
0b8092ed1
fdcf17476
0b0d70eb5
02d417bf2
da07a6a18
e9c303e76
060c67d30
f3e926578
f34138c18
f34fc12b6
04a713ebc
f35542bd8
eec9fecc7
d88c43ae1
03949460e
04928b31e
fc4b5280d
f7f6c1219
033aaae7e
f775f36a8
de969d016
d2a1b80c3
f91d92f39
03e7dd9c6
0533b9aa4
071e4e8b3
fd981666d
10fe33d64
e26d3de29
dcf2622af
0fed6ffb8
f6ab77d7a
f0fef68e5
ff0b719fd
f2f68ab9e
f95e5f0d2
df94a8ff0
ded8d19b2
ee00a835c
099bc38fa
082a9a358
032303b46
07ee77bf4
0fcb553be
ea4898078
dc92e8874
fa8b56143
fedd25fa1
13dd69fac
0f95f1a4e
06bcf193d
f2a754865
d839d0ab3
e8b5855a1
08c274a96
0b9c02b1a
103f47b9b
f945a24ab
0d1f72248
094fa70be
d3ed00be3
d8b16600c
f922bb79a
0389d42f5
024c12155
001d446a4
f88d70e6a
08089709f
d6663ec9c
deabc6b67
0cc6075f5
01e39f6b0
f38eae33c
0880f86c1
fa3aaeeff
0342c3fa8
cf9390790
f141d1b1a
fd546c865
02b200811
